## Tips and Hints
It is often easier to debug the code using the debugger and run the code in python instead of through the Jupyter file. From a new terminal, activate your virtual environment and run the unit test file (below assumes windows for activating the virtual environment):

```
.\mav_venv\Scripts\activate
python .\mav_sim\unit_tests\ch2_transforms_tests.py
```

To run the code within the mav sim from the terminal, you will activate your environment and run the book assignment for chapter 2
```
.\mav_venv\Scripts\activate
python .\book_assignments\mavsim_chap2.py
```

## Problem 1: Basic concentric frame transforms
Most of the frames have the same origin. The code in chap2/transforms.py calculates the rotation matrices used to transform between these frames. Correctly implement the following functions:
* rot_x: calculate elementary rotation matrix about x-axis
* rot_y: calculate elementary rotation matrix about y-axis
* rot_z: calculate elementary rotation matrix about z-axis
* rot_v_to_v1: calculates the rotation from frame v to v1
* rot_v1_to_v2: calculates the rotation from frame v1 to v2
* rot_v2_to_b: calculates the rotation from v2 to body frame
* rot_b_to_s: calculates the rotation from body to stability frame
* rot_s_to_w: calculates the rotation from stability to wind frame

*Hint:* You should only compute the cosine and sine of the angle in *rot_x*, *rot_y*, and *rot_z*. All the remaining functions should call those functions (i.e., one line change from what they currently are)

Use these function to compute the following. Assume that $\psi = \frac{\pi}{4}$, $\theta = 0.3$, $\phi = 0.25$, $\alpha = 0.1$, and $\beta = 0.15$. Display the results in the exported pdf.
* Compute $p_1^{v1}$ given $p_1^v = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$
* Compute $p_1^{w}$
* Compute $p_2^{s}$ given $p_2^{v2} = \begin{bmatrix} 4 \\ 5 \\ 6 \end{bmatrix}$
* Compute $p_2^v$



In [1]:
import numpy as np # Imports the numpy library and creates the alias np
from IPython.display import display # Used to display variables nicely in Jupyter
from mav_sim.chap2.transforms import rot_v_to_v1, rot_v1_to_v2, rot_v2_to_b, rot_b_to_s, rot_s_to_w

# Calculate the required rotation matrices
psi = np.pi/4
theta = 0.3
phi = 0.25
alpha = 0.1
beta = 0.15

# Initialize p1 and p2
p_1_v = np.array([[1],[2],[3]])
p_2_v2 = np.array([[4],[5],[6]])

# Calculate p_1^v1
p_1_v1 = rot_v_to_v1(psi) @ p_1_v
print('p_1^v1 = ')
display(p_1_v1)

# Calculate p_1^w
p_1_w = rot_v_to_v1(psi) @ rot_v1_to_v2(theta) @ rot_v2_to_b(phi) @ rot_b_to_s(alpha) @ rot_s_to_w(beta) @ p_1_v
print('p_1^w = ')
display(p_1_w)

# Calculate p_2^s
p_2_s = rot_v2_to_b(phi) @ rot_b_to_s(alpha) @ p_2_v2
print('p_2^s = ')
display(p_2_s)

# Calculate p_2^v
p_2_v = rot_v1_to_v2(theta).T @ rot_v_to_v1(psi).T @ p_2_v2
print('p_2^v = ')
display(p_2_v)


p_1^v1 = 


array([[2.12132034],
       [0.70710678],
       [3.        ]])

p_1^w = 


array([[2.33617133],
       [1.16821137],
       [2.67910166]])

p_2^s = 


array([[4.57901716],
       [6.2227732 ],
       [4.16049223]])

p_2^v = 


array([[1.09759633],
       [6.36396103],
       [5.94098328]])




## Problem 2: Compound rotation matrices
The transform from the vehicle frame to the body frame can be written as a compound of three rotation matrices (and so can the inverse transform). However, these matrices are used so often that it is nice to avoid multiplying these three matrices each time the transform is needed. 

Implement the following functions:

* rot_v_to_b: calculates the rotation from vehicle to body frame
* rot_b_to_v: calculates the rotation from body frame to vehicle frame

*Hint:* You really only need to implement one of them and then use a transpose for the other

Using the same values as above, show that your implementation produces the same rotation matrices as three elementary matrices multiplied together. Display the difference in the exported pdf.


In [2]:
from mav_sim.chap2.transforms import rot_v_to_b, rot_b_to_v

# Calculate the rotation matrices as compound rotation matrices (i.e., matrix multiplication)


# Calculate the rotation matrices using the functions


# Calculate and display the difference
R_v_to_b_diff = rot_v_to_b(psi, theta, phi) - (rot_v_to_v1(psi) @ rot_v1_to_v2(theta) @ rot_v2_to_b(phi))
print("R_v_to_b_diff")
display(R_v_to_b_diff)

R_b_to_v_diff = rot_b_to_v(psi, theta, phi) - (rot_v2_to_b(phi).T @ rot_v1_to_v2(theta).T @ rot_v_to_v1(psi).T)
print("R_b_to_v_diff")
display(R_b_to_v_diff)

R_v_to_b_diff


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

R_b_to_v_diff


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.42861287e-17,  0.00000000e+00,  0.00000000e+00]])

## Problem 3: Tranform to vehicle frame
Converting to and from the inertial frame requires translation. Implement the following functions:
* trans_i_to_v: transforms a point from inertial frame to the vehicle frame
* trans_v_to_i: transforms a point from vehicle frame to the inertial frame
* trans_i_to_b: transforms a point from inertial frame to the body frame
* trans_b_to_i: transforms a point from the body frame to the inertial frame

Note that the transform between inertial and the vehicle frame is purely translational. The transform between the vehicle and body frame is purely rotational. Thus, you can use the functions already implemented to make the *trans_i_to_b* and *trans_b_to_i* functions quite simple.

Given that the UAV is in the position $p_n = 1$, $p_e = 2$, and $p_d = 3$ with the angles defined as before, transform the following points to the body frame using the implemented functions:

$p_3^i = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

$p_4^i = \begin{bmatrix} 3 \\ 2 \\ 1\end{bmatrix}$

Transform the following point in the body frame to the inertial frame

$p_5^b = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

Transform the following point in the wind frame to the inertial frame

$p_6^w = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

Display the results in the exported pdf.

In [3]:
from mav_sim.chap2.transforms import trans_i_to_b, trans_b_to_i

# Create the pose of the aircraft
class Pose:
    def __init__(self) -> None:
        self.north: float     = 1.      # north position
        self.east: float      = 2.      # east position
        self.altitude: float  = 3.      # altitude
        self.phi: float       = phi     # roll angle
        self.theta: float     = theta   # pitch angle
        self.psi: float       = psi     # yaw angle
pose = Pose()

# Initialize the points
p_3_i = np.array([[1],[2],[3]])
p_4_i = np.array([[3],[2],[1]])
p_5_b = np.array([[1],[2],[3]])
p_6_w = np.array([[1],[2],[3]])

# Calculate p_3^b
p_3_b = trans_i_to_b(pose, p_3_i)
print("p_3^b = ")
display(p_3_b)

# Calculate p_4_b
p_4_b = trans_i_to_b(pose, p_4_i)
print("p_4^b = ")
display(p_4_b)

# Calcualte p_5^i
p_5_i = trans_b_to_i(pose, p_5_b)
print("p_5^i")
display(p_5_i)

# Calculate p_6^i
p_6_i = trans_b_to_i(pose, rot_s_to_w(beta).T @ rot_b_to_s(alpha).T @ p_6_w)
print("p_6^i")
display(p_6_i)


p_3^b = 


array([[-0.16516278],
       [ 2.26445498],
       [ 5.55382435]])

p_4^b = 


array([[1.2409413 ],
       [0.15858684],
       [4.29358998]])

p_5^i


array([[1.21103571],
       [3.29461294],
       [0.50420337]])

p_6^i


array([[0.76820094],
       [2.87583576],
       [0.63031416]])

## Static analysis
Run the static code analysis (you must have zero static code analysis errors to get credit). You may not modify the static code analysis configuration files.

### ISORT
Run Isort:
```
python -m isort mav_sim book_assignments
```

Terminal output (should be nothing):

### MyPy
Run MyPy
```
python -m mypy mav_sim/chap2/ book_assignments
```

Terminal output (should indicate no error):
```
Success: no issues found in 15 source files
```

### Pylint
Run Pylint
```
python -m pylint --jobs 0 --rcfile .pylintrc mav_sim/chap2/ book_assignments/
```

Terminal output (should indicate `10/10`)
```
Your code has been rated at 10.00/10
```

## Simple code checking
The following code does not need to change. It should just be used as a sanity check so that you know the code is implemented properly. The output should not have any lines reading `Failed test!`

In [4]:
from mav_sim.unit_tests.ch2_transforms_tests import run_all_tests, SimpleRotationTest, ThreeRotationTest, TransformTest
run_all_tests()


Running rot_x_tests...
rot_x_tests passed

Running rot_y_tests...
rot_y_tests passed

Running rot_z_tests...
rot_z_tests passed

Running rot_v_to_v1_tests...
rot_v_to_v1_tests passed

Running rot_v1_to_v2_tests...
rot_v1_to_v2_tests passed

Running rot_v2_to_b_tests...
rot_v2_to_b_tests passed

Running rot_b_to_s_tests...
rot_b_to_s_tests passed

Running rot_s_to_w_tests...
rot_s_to_w_tests passed

Running rot_v_to_b_tests...


Failed test!
test: 
 Input: phi=9.870642132904692, theta=13.552486288288279, psi=5.233229837452011
Desired output:
[[ 2.74647255e-01 -4.78747369e-01 -8.33888387e-01]
 [-9.61544995e-01 -1.37041515e-01 -2.38014382e-01]
 [-3.28568274e-04  8.67191201e-01 -4.97975213e-01]] 
actual result: 
 [[ 0.27464726  0.60366     0.7484407 ]
 [ 0.47874737 -0.76088354  0.43801507]
 [ 0.83388839  0.23801438 -0.49797521]]

Running rot_b_to_v_tests...


Failed test!
test: 
 Input: phi=8.60531387772917, theta=15.003260208400427, psi=7.711359101104929
Desired output:
[[-0.10828244  0.